In [1]:
import os
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10, mnist, fashion_mnist
import numpy as np
import time
import math
import random

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [2]:
# fine-tuning and weight pruning attacks. 
from tune import FTLL, FTAL, RTAL, Prune 
# extraction attacks. 
from extraction import BlackboxModel, KnockoffClient, JBAClient, ESAClient 

In [3]:
# load and pre-process the mnist dataset. 
num_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [4]:
# sample a subset of training data for tuning.
NUMS = 10000
idx = np.random.choice(30000, NUMS, replace=False) + 30000
x_sub, y_sub = x_train[idx], y_train[idx]

In [5]:
victim_model = load_model('./example/mnist_lenet5.h5')
tuned_model = FTAL(victim_model, x_sub, y_sub, aug=True)
# tuned_model.evaluate(x_test, y_test)

FTAL: Using real-time data augmentation.
Done


In [6]:
victim_model = load_model('./example/mnist_lenet5.h5')
pruned_model = Prune(victim_model, x_sub, y_sub, r=0.6, aug=True) # weight pruning rate 60%

Prune: Using real-time data augmentation.
Done


In [7]:
import sys
sys.path.append('..')
from utils.lenet5 import Lenet5

victim_model = load_model('./example/mnist_lenet5.h5')
blackbox_model = BlackboxModel(victim_model) 

In [8]:
# JBA extraction
SEED_SIZE = 150
idx = np.random.choice(30000, size=SEED_SIZE, replace=False)
x_seeds = x_train[idx]

substitute_model = Lenet5((28,28,1), 10)
jba = JBAClient(x_seeds=x_seeds, extract_round=6, scale_const=0.01, train_epoch=10)
extracted_model = jba.extract(substitute_model, blackbox_model)
# extracted_model.evaluate(x_test, y_test)

In [9]:
# ESA extraction
substitute_model = Lenet5((28,28,1), 10)
esa = ESAClient(extract_epoch=10, syns_num=20000, syns_epoch=30, syns_step=0.01, train_epoch=10)
extracted_model = esa.extract(substitute_model, blackbox_model)
# extracted_model.evaluate(x_test, y_test)

    Total Extraction Epochs: 10
    Synthesis X size: (20000, 28, 28, 1)


In [10]:
# Knockoff extraction
(x_train_sub, y_train_sub), (x_test_sub, y_test_sub) = fashion_mnist.load_data()
x_train_sub = x_train_sub.astype('float32') / 255
x_train_sub = np.reshape(x_train_sub, (len(x_train_sub), 28, 28, 1))

substitute_model = Lenet5((28,28,1), 10)
ko = KnockoffClient(x_sub=x_train_sub, size=0.5, train_epoch=10)
extracted_model = ko.extract(substitute_model, blackbox_model)
# extracted_model.evaluate(x_test, y_test)

    Random sampling size: 30000
